# r4pm - Process Mining Bindings

This notebook demonstrates the key features of r4pm:
- Auto-generated bindings with full IDE support
- Automatic type conversion (no manual conversion needed!)
- Registry-based data management
- Fast XES/OCEL import/export

## 1. Basic Example: Automatic Type Conversion

The key feature - pass any data type to functions, conversion happens automatically!

In [1]:
import r4pm.bindings as bindings
import r4pm

# Load an OCEL file - returns a registry ID
ocel_id = r4pm.import_item('OCEL', 'test_data/order-management.xml')
print(f"✓ Loaded OCEL: {ocel_id[:20]}...")

# Use it directly - automatic conversion!
# num_events expects IndexLinkedOCEL, but we pass OCEL - it converts automatically
num = bindings.num_events(ocel=ocel_id)
print(f"✅ Number of events: {num}")

# Check registry - now has 2 items (original + converted)
items = r4pm.list_items()
print(f"\n📋 Registry now has {len(items)} items:")
for item in items:
    print(f"   {item['type']}: {item['id'][:30]}...")

✓ Loaded OCEL: ab428ab0-e51d-42bb-a...
✅ Number of events: 21008

📋 Registry now has 2 items:
   OCEL: ab428ab0-e51d-42bb-affe-f7e651...
   SlimLinkedOCEL: ab428ab0-e51d-42bb-affe-f7e651...


In [2]:
r4pm.bindings.analysis.object_centric.get_object_attribute_changes(ocel_id,"Echo")

{'traces': {'price': [{'time': '2023-04-03T01:00:00+02:00', 'value': 99.99},
   {'time': '2023-06-02T11:39:36+02:00', 'value': 105.7},
   {'time': '2023-08-01T14:17:59+02:00', 'value': 108.19},
   {'time': '2023-09-30T16:24:43+02:00', 'value': 109.76},
   {'time': '2023-12-01T10:01:30+01:00', 'value': 112.39},
   {'time': '2024-01-30T11:45:23+01:00', 'value': 117.85}],
  'weight': [{'time': '2023-04-03T01:00:00+02:00', 'value': 0.78}]}}

In [3]:
item_log = r4pm.bindings.flatten_ocel_on(ocel_id,"items")
# Create a dotted chart based o nthe flattened item log
item_log_dotted = r4pm.bindings.analysis.case_centric.get_dotted_chart(item_log)
# See how many dots there are
print(len(item_log_dotted['y_values']))

7659


In [4]:
socel_id = r4pm.import_item('SlimLinkedOCEL', 'test_data/order-management.xml')
res = r4pm.bindings.discovery.object_centric.oc_declare.discover_oc_declare(socel_id)
print(f"\n🔍 OC-Declare discovery result:\n{res}")


🔍 OC-Declare discovery result:
[{'arc_type': 'AS', 'counts': [1, None], 'from': 'pay order', 'label': {'all': [], 'any': [{'object_type': 'products', 'type': 'Simple'}], 'each': [{'object_type': 'items', 'type': 'Simple'}]}, 'to': 'pick item'}, {'arc_type': 'AS', 'counts': [1, None], 'from': 'pay order', 'label': {'all': [], 'any': [{'object_type': 'items', 'type': 'Simple'}], 'each': [{'object_type': 'products', 'type': 'Simple'}]}, 'to': 'pick item'}, {'arc_type': 'DP', 'counts': [1, None], 'from': 'package delivered', 'label': {'all': [{'object_type': 'items', 'type': 'Simple'}, {'object_type': 'products', 'type': 'Simple'}], 'any': [], 'each': [{'object_type': 'employees', 'type': 'Simple'}, {'object_type': 'packages', 'type': 'Simple'}]}, 'to': 'send package'}, {'arc_type': 'EP', 'counts': [1, None], 'from': 'pay order', 'label': {'all': [{'object_type': 'items', 'type': 'Simple'}, {'object_type': 'products', 'type': 'Simple'}], 'any': [], 'each': [{'object_type': 'orders', 'type

## 2. Process Discovery

All discovery algorithms work with automatic conversion

In [5]:
# DFG from OCEL (auto-converts to SlimLinkedOCEL)
dfg = bindings.discover_dfg_from_ocel(ocel_id)
object_types = list(dfg['object_type_to_dfg'].keys())
print(f"✓ DFG for {len(object_types)} object types: {', '.join(object_types)}")

# OC-Declare (needs SlimLinkedOCEL - load directly)
slim_id = r4pm.import_item('SlimLinkedOCEL', 'test_data/order-management.xml')
options = {
    "noise_threshold": 0.0,
    "o2o_mode": "None",
    "counts_for_generation": [1, None],
    "counts_for_filter": [1, None],
    "reduction": "None",
    "refinement": False,
    "considered_arrow_types": ["AS"]
}
constraints = bindings.discover_oc_declare(slim_id, options)
print(f"✓ Discovered {len(constraints)} OC-Declare constraints")

✓ DFG for 6 object types: customers, employees, items, orders, packages, products
✓ Discovered 154 OC-Declare constraints


## 3. Registry Operations

Manual conversion and export when needed

In [6]:
# Get as DataFrames
ocel_dfs = r4pm.item_to_df(ocel_id)
print(f"✓ DataFrames: {list(ocel_dfs.keys())}")
print(f"  Events: {ocel_dfs['events'].shape}")
print(f"  Objects: {ocel_dfs['objects'].shape}")

# Export to file
r4pm.export_item(ocel_id, "test_data/output.xml")
print(f"\n✓ Exported OCEL XML")
r4pm.export_item(ocel_id, "test_data/output.json")
print(f"\n✓ Exported OCEL JSON")

# Clean up
for item in r4pm.list_items():
    r4pm.remove_item(item['id'])
print("✓ Registry cleaned")

✓ DataFrames: ['objects', 'relations', 'object_changes', 'events', 'o2o']
  Events: (21008, 3)
  Objects: (10840, 2)

✓ Exported OCEL XML

✓ Exported OCEL JSON
✓ Registry cleaned


## 4. Simple Import/Export API - Direct DataFrame Import

For direct DataFrame operations without registry

In [7]:
# XES import - returns (DataFrame, attributes_json)
xes, attrs = r4pm.df.import_xes("test_data/Sepsis Cases - Event Log.xes.gz")
print(f"✓ XES shape: {xes.shape}")

# OCEL import - returns dict of DataFrames
ocel = r4pm.df.import_ocel_xml("test_data/order-management.xml")
print(f"✓ OCEL events: {ocel['events'].shape}")

# Export
r4pm.df.export_xes(xes, "test_data/output.xes")
print("✓ Exported XES")

✓ XES shape: (15214, 32)
✓ OCEL events: (21008, 3)
✓ Exported XES


In [8]:
r4pm.bindings.test_some_inputs("Hello",1,3,3.333,True)

's=Hello,n=1,i=3,f=3.333,b=true'

In [9]:
for f in r4pm.list_bindings():
    print(f['name'])

discover_alpha+++
locel_add_object_type
locel_get_ob_type
locel_get_obs_of_type
locel_new
locel_get_ob_types
locel_get_ev_type
locel_add_e2o
locel_get_ob_type_of
locel_get_e2o_rev
locel_add_object
locel_get_ob_id
locel_get_ev_attr_val
locel_delete_o2o
locel_get_ev_type_of
locel_get_e2o
locel_get_ev_time
locel_get_o2o
locel_delete_e2o
locel_construct_ocel
locel_add_event_type
locel_get_ev_types
locel_get_full_ob
locel_get_ev_id
locel_get_evs_of_type
locel_get_ev_by_id
locel_add_event
locel_get_full_ev
locel_get_ob_attr_vals
locel_add_o2o
locel_get_o2o_rev
locel_get_ob_by_id
get_event_timestamps
discover_dfg_from_ocel
add_init_exit_events_to_ocel
log_to_activity_projection
discover_oc_declare
oc_declare_conformance
discover_dfg
get_object_attribute_changes
get_dotted_chart
flatten_ocel_on
test_some_inputs
index_link_ocel
slim_link_ocel
ocel_type_stats
num_objects
num_events
